<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [29]:
!pip3 install beautifulsoup4
!pip3 install requests

In [30]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [31]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [32]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [33]:
import requests

url = 'https://en.wikipedia.org/wiki/List_of_Falcon_9_launches'

response = requests.get(url)

if response.status_code == 200:
    print("Request was successful!")
    
    print(response.content[:500])  # Printing only the first 500 characters for brevity
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Request was successful!
b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-'


import requests
from bs4 import BeautifulSoup

# Step 1: Set the URL for the Falcon 9 Launch Wiki page
url = 'https://en.wikipedia.org/wiki/List_of_Falcon_9_launches'

# Step 2: Perform the GET request
response = requests.get(url)

# Step 3: Check if the request was successful (status code 200)
if response.status_code == 200:
    print("Request was successful!")

    # Step 4: Create a BeautifulSoup object from the HTML response
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Step 5: Optionally, print the first 500 characters of the parsed HTML
    print(soup.prettify()[:500])  # Printing only the first 500 characters for brevity
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


In [34]:
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_Falcon_9_launches'

response = requests.get(url)

if response.status_code == 200:
    print("Request was successful!")

    soup = BeautifulSoup(response.content, 'html.parser')

    print(soup.prettify()[:500])  # Printing only the first 500 characters for brevity
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

Request was successful!
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [35]:
url = 'https://en.wikipedia.org/wiki/List_of_Falcon_9_launches'

response = requests.get(url)

if response.status_code == 200:
    print("Request was successful!")

    soup = BeautifulSoup(response.content, 'html.parser')

    page_title = soup.title.string
    print(f"Page Title: {page_title}")

    print(soup.prettify()[:500])
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

Request was successful!
Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [36]:
url = 'https://en.wikipedia.org/wiki/List_of_Falcon_9_launches'

response = requests.get(url)

if response.status_code == 200:
    print("Request was successful!")

    soup = BeautifulSoup(response.content, 'html.parser')
    
    tables = soup.find_all('table')

    if tables:
        header_row = tables[0].find('tr')  # Get the first row of the first table
        headers = [th.text.strip() for th in header_row.find_all('th')]  # Extract headers
        
        print("Column Names:")
        for header in headers:
            print(header)
    else:
        print("No tables found on the page.")

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

Request was successful!
Column Names:
Flight No.
Date andtime (UTC)
Version,booster[f]
Launchsite
Payload[g]
Payload mass
Orbit
Customer
Launchoutcome
Boosterlanding


Starting from the third table is our target table contains the actual launch records.


In [38]:
pip install requests pandas beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [40]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Passo 1: Fazer a requisição à página
url = 'https://en.wikipedia.org/wiki/List_of_Falcon_9_launches'  # Substitua pela URL real da página que contém as tabelas
response = requests.get(url)

# Passo 2: Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    # Passo 3: Criar um objeto BeautifulSoup para analisar o conteúdo
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Passo 4: Extrair todas as tabelas da página
    html_tables = soup.find_all('table')
    
    # Passo 5: Converter as tabelas em DataFrames do pandas
    tables = [pd.read_html(str(table))[0] for table in html_tables]

    # Passo 6: Acessar a terceira tabela (índice 2)
    if len(tables) > 2:  # Verificar se existem pelo menos 3 tabelas
        third_table = tables[2]
        print(third_table)
    else:
        print("Menos de 3 tabelas encontradas.")
else:
    print("Erro ao acessar a página. Código de status:", response.status_code)


   Date and time (UTC)[400]  \
0      21 October 202422:58   
1      21 October 202422:58   
2      24 October 202417:15   
3      24 October 202417:15   
4              October 2024   
5              October 2024   
6              October 2024   
7              October 2024   
8      2 November 202403:40   
9      2 November 202403:40   
10            November 2024   
11            November 2024   
12            November 2024   
13            November 2024   
14            November 2024   
15            November 2024   
16       November 2024[411]   
17       November 2024[411]   
18       November 2024[414]   
19       November 2024[414]   
20            November 2024   
21            November 2024   
22            November 2024   
23                2024[417]   
24                2024[417]   
25                2024[421]   
26                2024[421]   
27                2024[424]   
28                2024[424]   
29                2024[417]   
30                2024[417]   
31      

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [41]:
url = 'https://en.wikipedia.org/wiki/List_of_Falcon_9_launches'

response = requests.get(url)

if response.status_code == 200:
    print("Request was successful!")

    soup = BeautifulSoup(response.content, 'html.parser')

    tables = soup.find_all('table')

    if tables:
        first_launch_table = tables[0]  

        def extract_column_from_header(th):
            return th.text.strip() if th.text.strip() else None

        column_names = []

        header_row = first_launch_table.find('tr')
        for th in header_row.find_all('th'):
            name = extract_column_from_header(th)
            if name is not None and len(name) > 0:
                column_names.append(name)  

        print("Column Names:")
        for column in column_names:
            print(column)
    else:
        print("No tables found on the page.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

Request was successful!
Column Names:
Flight No.
Date andtime (UTC)
Version,booster[f]
Launchsite
Payload[g]
Payload mass
Orbit
Customer
Launchoutcome
Boosterlanding


Check the extracted column names


In [42]:
print(column_names)

['Flight No.', 'Date andtime (UTC)', 'Version,booster[f]', 'Launchsite', 'Payload[g]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [54]:
print(launch_dict.keys())


# Vamos supor que o nome correto seja 'Date and time' (ajuste conforme necessário)
column_to_remove = 'Date and time'  # Use o nome correto da coluna
if column_to_remove in launch_dict:
    del launch_dict[column_to_remove]
else:
    print(f"A coluna '{column_to_remove}' não existe no dicionário.")

# Continue inicializando os valores do launch_dict
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Adicionando novas colunas
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []


dict_keys(['Flight No.', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Version Booster', 'Booster landing', 'Date', 'Time'])
A coluna 'Date and time' não existe no dicionário.


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [55]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)
            

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [56]:
import pandas as pd

# Inicialização do dicionário
launch_dict = {
    'Flight No.': [],
    'Launch site': [],
    'Payload': [],
    'Payload mass': [],
    'Orbit': [],
    'Customer': [],
    'Launch outcome': [],
    'Version Booster': [],
    'Booster landing': [],
    'Date': [],
    'Time': []
}

# Criação do DataFrame
df = pd.DataFrame({key: pd.Series(value, dtype='object') for key, value in launch_dict.items()})

# Verificando o DataFrame
print(df)


Empty DataFrame
Columns: [Flight No., Launch site, Payload, Payload mass, Orbit, Customer, Launch outcome, Version Booster, Booster landing, Date, Time]
Index: []


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


In [57]:
first_static_fire_date = data['static_fire_date_utc'].iloc[0]
year = pd.to_datetime(first_static_fire_date).year
print(year)


TypeError: list indices must be integers or slices, not str

In [58]:
falcon9_count = data[data['BoosterVersion'].str.contains('Falcon 9')].shape[0]
print(falcon9_count)


TypeError: list indices must be integers or slices, not str

In [51]:
import requests
import pandas as pd

# Passo 1: Requisição GET na API do SpaceX
url = 'https://api.spacexdata.com/v4/launches'
response = requests.get(url)

if response.status_code == 200:
    # Passo 2: Converter a resposta em um DataFrame
    launches_data = response.json()
    data = pd.json_normalize(launches_data)
    
    # Agora você pode realizar as operações a seguir com o DataFrame 'data'
    
    # Passo 3: Contar lançamentos do Falcon 9
    falcon9_count = data[data['rocket'].str.contains('Falcon 9')].shape[0]
    print("Número de lançamentos do Falcon 9:", falcon9_count)
    
    # Passo 4: Contar valores faltantes na coluna landingPad
    missing_landing_pad = data['landing_pad'].isnull().sum()
    print("Valores faltantes na coluna landingPad:", missing_landing_pad)
    
    # Passo 5: Para verificar o título da página Wiki do Falcon 9
    from bs4 import BeautifulSoup

    wiki_url = 'https://en.wikipedia.org/wiki/List_of_Falcon_9_launches'
    wiki_response = requests.get(wiki_url)

    if wiki_response.status_code == 200:
        soup = BeautifulSoup(wiki_response.content, 'html.parser')
        title = soup.title.string
        print("Título da página Wiki:", title)
else:
    print("Erro ao acessar a API. Código de status:", response.status_code)


Número de lançamentos do Falcon 9: 0


KeyError: 'landing_pad'

In [59]:
import pandas as pd
import requests

# Passo 1: Fazer a requisição à API do SpaceX
response = requests.get('https://api.spacexdata.com/v4/launches')
data = response.json()

# Passo 2: Criar o DataFrame
df = pd.json_normalize(data)

# Passo 3: Verificar os nomes das colunas
print(df.columns)


Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')


In [60]:
# Passo 4: Contar valores faltantes na coluna launchpad
missing_landing_pad = df['launchpad'].isnull().sum()
print("Valores faltantes na coluna launchpad:", missing_landing_pad)


Valores faltantes na coluna launchpad: 0


In [ ]:
KeyError                                  Traceback (most recent call last)
~/conda/envs/python/lib/python3.7/site-packages/pandas/core/indexes/base.py in get_loc(self, key, method, tolerance)
   3360             try:
-> 3361                 return self._engine.get_loc(casted_key)
   3362             except KeyError as err:

~/conda/envs/python/lib/python3.7/site-packages/pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc()

~/conda/envs/python/lib/python3.7/site-packages/pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc()

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item()

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item()

KeyError: 'landing_pad'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
/tmp/ipykernel_1227/3056469927.py in <module>
     18 
     19     # Passo 4: Contar valores faltantes na coluna landingPad
---> 20     missing_landing_pad = data['landing_pad'].isnull().sum()
     21     print("Valores faltantes na coluna landingPad:", missing_landing_pad)
     22 

~/conda/envs/python/lib/python3.7/site-packages/pandas/core/frame.py in __getitem__(self, key)
   3456             if self.columns.nlevels > 1:
   3457                 return self._getitem_multilevel(key)
-> 3458             indexer = self.columns.get_loc(key)
   3459             if is_integer(indexer):
   3460                 indexer = [indexer]

~/conda/envs/python/lib/python3.7/site-packages/pandas/core/indexes/base.py in get_loc(self, key, method, tolerance)
   3361                 return self._engine.get_loc(casted_key)
   3362             except KeyError as err:
-> 3363                 raise KeyError(key) from err
   3364 
   3365         if is_scalar(key) and isna(key) and not self.hasnans:

KeyError: 'landing_pad'


In [52]:
import pandas as pd
import requests

# Passo 1: Fazer a requisição à API do SpaceX
response = requests.get('https://api.spacexdata.com/v4/launches')
data = response.json()

# Passo 2: Criar o DataFrame
df = pd.json_normalize(data)

# Passo 3: Filtrar lançamentos do Falcon 9
falcon_9_launches = df[df['rocket'] == 'Falcon 9']

# Passo 4: Contar os lançamentos do Falcon 9
count_falcon_9 = len(falcon_9_launches)
print("Total de lançamentos do Falcon 9:", count_falcon_9)


Total de lançamentos do Falcon 9: 0


In [53]:
# Passo 5: Contar valores faltantes na coluna landingPad
missing_landing_pad = df['launchpad'].isnull().sum()  # Corrigi para 'launchpad', que é o nome correto
print("Valores faltantes na coluna landingPad:", missing_landing_pad)


Valores faltantes na coluna landingPad: 0


In [22]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Passo 1: Requisição GET na API do SpaceX
url = 'https://api.spacexdata.com/v4/launches'
response = requests.get(url)

if response.status_code == 200:
    # Passo 2: Converter a resposta em um DataFrame
    launches_data = response.json()
    data = pd.json_normalize(launches_data)

    # Exibir as colunas para verificar os nomes
    print("Colunas disponíveis:", data.columns)
    
    # Passo 3: Contar lançamentos do Falcon 9
    falcon9_count = data[data['rocket'].str.contains('Falcon 9')].shape[0]
    print("Número de lançamentos do Falcon 9:", falcon9_count)
    
    # Passo 4: Contar valores faltantes na coluna launchpad
    missing_launch_pad = data['launchpad'].isnull().sum()  # Alterado para 'launchpad'
    print("Valores faltantes na coluna launchpad:", missing_launch_pad)
    
    # Passo 5: Para verificar o título da página Wiki do Falcon 9
    wiki_url = 'https://en.wikipedia.org/wiki/List_of_Falcon_9_launches'
    wiki_response = requests.get(wiki_url)

    if wiki_response.status_code == 200:
        soup = BeautifulSoup(wiki_response.content, 'html.parser')
        title = soup.title.string
        print("Título da página Wiki:", title)
else:
    print("Erro ao acessar a API. Código de status:", response.status_code)


Colunas disponíveis: Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
Número de lançamentos do Falcon 9: 0
Valores faltantes na coluna launchpad: 0
Título da página Wiki: List of Falcon 9 and Falcon Heavy launches - Wikipedia


In [19]:
title = soup.title
print(title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


In [23]:
first_year = pd.to_datetime(data['static_fire_date_utc'].iloc[0]).year
print("Ano da primeira linha da coluna static_fire_date_utc:", first_year)


Ano da primeira linha da coluna static_fire_date_utc: 2006


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


In [24]:
import requests
import pandas as pd

# Passo 1: Requisição GET na API do SpaceX
url = 'https://api.spacexdata.com/v4/launches'
response = requests.get(url)

if response.status_code == 200:
    # Passo 2: Converter a resposta em um DataFrame
    launches_data = response.json()
    data = pd.json_normalize(launches_data)

    # Passo 3: Obter o ano da primeira linha da coluna static_fire_date_utc
    first_year = pd.to_datetime(data['static_fire_date_utc'].iloc[0]).year
    print("Ano da primeira linha da coluna static_fire_date_utc:", first_year)
else:
    print("Erro ao acessar a API. Código de status:", response.status_code)


Ano da primeira linha da coluna static_fire_date_utc: 2006


<!--
## Change Log
-->


In [25]:
import requests
import pandas as pd

# Passo 1: Requisição GET na API do SpaceX
url = 'https://api.spacexdata.com/v4/launches'
response = requests.get(url)

if response.status_code == 200:
    # Passo 2: Converter a resposta em um DataFrame
    launches_data = response.json()
    data = pd.json_normalize(launches_data)

    # Passo 3: Contar lançamentos do Falcon 9 (excluindo o Falcon 1)
    falcon9_count = data[data['rocket'].str.contains('Falcon 9')].shape[0]
    print("Número de lançamentos do Falcon 9:", falcon9_count)
else:
    print("Erro ao acessar a API. Código de status:", response.status_code)


Número de lançamentos do Falcon 9: 0


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


In [26]:
import requests
import pandas as pd

# Passo 1: Requisição GET na API do SpaceX
url = 'https://api.spacexdata.com/v4/launches'
response = requests.get(url)

if response.status_code == 200:
    # Passo 2: Converter a resposta em um DataFrame
    launches_data = response.json()
    data = pd.json_normalize(launches_data)

    # Passo 3: Contar valores faltantes na coluna landingPad
    # A coluna correta pode ser 'launchpad', mas verifique se está certo no seu caso
    missing_landing_pad = data['launchpad'].isnull().sum()  # Use 'landing_pad' se existir
    print("Valores faltantes na coluna landingPad:", missing_landing_pad)
else:
    print("Erro ao acessar a API. Código de status:", response.status_code)


Valores faltantes na coluna landingPad: 0


Copyright © 2021 IBM Corporation. All rights reserved.


In [27]:
import requests
import pandas as pd

# Passo 1: Requisição GET na API do SpaceX
url = 'https://api.spacexdata.com/v4/launches'
response = requests.get(url)

if response.status_code == 200:
    # Passo 2: Converter a resposta em um DataFrame
    launches_data = response.json()
    data = pd.json_normalize(launches_data)

    # Passo 3: Verificar o conteúdo da coluna 'rocket'
    print("Conteúdo da coluna 'rocket':", data['rocket'].unique())  # Verifique quais valores estão aqui

    # Passo 4: Contar lançamentos do Falcon 9
    falcon9_count = data[data['rocket'].str.contains('Falcon 9', na=False)].shape[0]
    print("Número de lançamentos do Falcon 9:", falcon9_count)
else:
    print("Erro ao acessar a API. Código de status:", response.status_code)


Conteúdo da coluna 'rocket': ['5e9d0d95eda69955f709d1eb' '5e9d0d95eda69973a809d1ec'
 '5e9d0d95eda69974db09d1ed']
Número de lançamentos do Falcon 9: 0


In [28]:
import requests
import pandas as pd

# Passo 1: Requisição GET na API do SpaceX
url = 'https://api.spacexdata.com/v4/launches'
response = requests.get(url)

if response.status_code == 200:
    # Passo 2: Converter a resposta em um DataFrame
    launches_data = response.json()
    data = pd.json_normalize(launches_data)

    # Passo 3: Verificar o conteúdo da coluna 'launchpad'
    print("Conteúdo da coluna 'launchpad':", data['launchpad'].isnull().sum())  # Verifique se existem valores nulos

    # Passo 4: Contar valores faltantes na coluna launchpad
    missing_landing_pad = data['launchpad'].isnull().sum()
    print("Valores faltantes na coluna launchpad:", missing_landing_pad)
else:
    print("Erro ao acessar a API. Código de status:", response.status_code)


Conteúdo da coluna 'launchpad': 0
Valores faltantes na coluna launchpad: 0
